In [1]:
# import sys  
# from pathlib import Path
# sys.path[0] = str(Path(sys.path[0]).parent)
# print(sys.path)

In [29]:
# Imports

from loguru import logger
from collections import defaultdict
from src.outlier_detection_methods import FewShotDetector, NaiveAggregator, local_knn
from src.utils.utils import (
    set_random_seed, load_model, merge_from_dict
)
from src.augmentations import __dict__ as AUGMENTATIONS
from typing import Dict
from types import SimpleNamespace
import yaml
import numpy as np
import matplotlib.pyplot as plt
from torch import Tensor
from pathlib import Path
from mpl_toolkits.axes_grid1 import ImageGrid
from src.utils.data_fetchers import get_task_loader, get_test_features
import torch

In [2]:
# Variables

dataset = "mini_imagenet"
n_way = 5
n_shot = 1
n_query = 10
n_tasks = 100
backbone = 'resnet12'
training = 'feat'
layers='4_2'

In [3]:
# Setup

logger.info("Loading data...")
data_loader = get_task_loader(dataset,
                              n_way,
                              n_shot,
                              n_query,
                              n_tasks,
                              split="test",
                              n_workers=6)

# average_train_features = {}
# std_train_features = {}
# for layer in layers.split('-'):
#     _, _, average_train_features[layer], std_train_features[layer] = get_test_features(
#         backbone, dataset, training, layer)



2022-02-02 19:00:14.290 | INFO     | __main__:<module>:3 - Loading data...
100%|█████████████████████████████████████████████████████| 12000/12000 [00:00<00:00, 5234156.41it/s]


In [40]:
logger.info("Building model...")
weights = Path('data') / 'models' / f'{backbone}_{dataset}_{training}.pth'
feature_extractor = load_model(backbone=backbone,
                               weights=weights,
                               dataset_name=dataset,
                               device='cuda')

logger.info("Loading mean/std from base set ...")

2022-02-02 19:14:12.648 | INFO     | __main__:<module>:1 - Building model...
2022-02-02 19:14:12.649 | INFO     | src.utils.utils:load_model:110 - Fetching data...
100%|█████████████████████████████████████████████████████| 38400/38400 [00:00<00:00, 5170174.42it/s]
2022-02-02 19:14:13.296 | INFO     | src.utils.utils:load_model:113 - Building model...
2022-02-02 19:14:13.456 | INFO     | __main__:<module>:8 - Loading mean/std from base set ...


Loaded weights from data/models/resnet12_mini_imagenet_feat.pth
Missing keys []
Unexpected keys []


In [ ]:
for support, support_labels, query, query_labels, _ in data_loader:
    plot_grid(support)
    augmented = AUGMENTATIONS['cutmix'](support)
    break

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [42]:
def plot_grid(images: Tensor):
    fig = plt.figure(figsize=(15, 75))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(1, 5),  # creates 2x2 grid of axes
                     axes_pad=0.3,  # pad between axes in inch.
                     )
    std = torch.Tensor([0.229, 0.224, 0.225]).to(images.device).view(3, 1, 1)
    mean = torch.Tensor([0.485, 0.456, 0.406]).to(images.device).view(3, 1, 1)
    shifted = images * std + mean
    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im.permute(1, 2, 0))